In [ ]:
import sys
# Hay que ejecutar esta línea antes de importar el módulo.
#sys.path.append("C:/Users/Vicen/PycharmProjects/tgpy")
# Ahora se puede importar el módulo.
%reload_ext tgpy
import tgpy as tg
import matplotlib.pyplot as plt
import mpl_scatter_density
import utm
import pydeck as pdk
import pandas as pd
import numpy as np
import seaborn as sb
import random
from scipy import signal
from sklearn.metrics import mean_absolute_error

In [ ]:
sb.set_context('notebook', font_scale=1.4)
sb.set_style('ticks')
plt.rcParams['figure.figsize'] = (12, 5)



# Funcion de extrarccion de parametros:

In [ ]:
def get_priors_dt(self, npriors: int):
    """
    Save the value of sample in a dict

    :param npriors: an int, the number of groups
    :return: a dict, value of sample for each prior and group
    """
    prior_dict = {}
    for i in range(npriors):
            prior_dict[('prior{}'.format(i), 'Year')] = self._priors_dict['prior{}'.format(i)].p[
                'Year'].data.clone().detach().cpu().numpy() # se ha adecuar a los como se guardo el dict de los parametros
    return prior_dict

# Entranamiento de un GP: 

In [ ]:
Sunspots = pd.read_csv('Sunspots.csv')
Sunspots['year'] = pd.DatetimeIndex(Sunspots['Date']).year
Sunspots_train_anual = Sunspots[['year','Monthly Mean Total Sunspot Number']]
Sunspots_train_anual = Sunspots_train_anual.groupby("year", as_index=False).mean()
Sunspots_train_anual = Sunspots_train_anual.iloc[51:272]
(Sunspots_train_anual).reset_index(inplace=True, drop=True)
S_train = Sunspots_train_anual

In [ ]:
S_train['year']

In [ ]:
S_train['year'] = S_train['year']-S_train[
    'year'].min()
S_train

In [ ]:


t_anual = S_train["year"]-S_train["year"]
y_anual = S_train["Monthly Mean Total Sunspot Number"]
prop = 0.8
obs = int(prop*len(t_anual))
train_index_anual = list(range(obs))
valid_index_anual = list(range(obs,len(t_anual)))
t_obs_anual = t_anual[train_index_anual]
y_obs_anual = y_anual[train_index_anual]
t_val = t_anual[valid_index_anual]
y_val = y_anual[valid_index_anual]



In [ ]:
valid_index_anual


In [ ]:
S_train['Monthly Mean Total Sunspot Number'].mean()

In [ ]:
y_obs_anual

In [ ]:
print(len(t_anual),len(t_obs_anual))

In [ ]:
def train(tgp, sample_priors=True, niters=1000):
    if sample_priors:
        tgp.sample_priors()
        tgp.plot_priors(kde=True)
    learning = tg.TgLearning(tgp, lr=0.01, pbatch=0.8) #rand_pert=0.0
    learning.execute_sgd(niters)
    tgp.plot_priors(kde=True)

In [ ]:
#train_index_anual = sorted(tg.np.random.choice(train_index_anual, 100))

## Modelo Kernel SE

In [ ]:
def model_se(df, index_obs, dim = 100): 
    inputs = ['year']
    outputs = ["Monthly Mean Total Sunspot Number"]
    dt = tg.DataTensor(df[inputs + outputs], inputs=inputs, outputs=outputs)
    dt.calculate_scale(inputs=True, outputs=False, quantile=False)
    
    var_se = tg.TgPrior('var', ['Sunspots'], dim=dim, low=0.001, high=1, alpha=4, beta=2)
    relevance = tg.TgPrior('relevance', ['year'], dim=dim, low=1, high=10, alpha=2, beta=4) #low = 0.0, high = 2
    noise = tg.TgPrior('noise', ['noise'], dim=dim, low=0.0, high=0.1, alpha=2, beta=4)
    shift = tg.TgPrior('shift', ['Sunspots'], dim=dim, low=-0.01, high=0.01, alpha=2, beta=4)
    power = tg.TgPrior('power', ['Sunspots'], dim=dim, low=0.01, high=1.2, alpha=2, beta=4)
    
    kernel = tg.SE(var_se, relevance)
    cov = tg.CovarianceTransport(kernel, noise=tg.WN(noise))
    marginal = tg.MarginalTransport(tg.BoxCoxShift(power, shift) )
    positive = tg.MarginalTransport(tg.Relu())
    
    tgp = tg.TGP([cov,marginal,positive], dt=dt)
    tgp.obs(index_obs)
    return tgp

tgp_se = model_se(S_train, train_index_anual)

In [ ]:
tgp_se.plot_predict('Sunspots Gaussian Process SE', 'year', "Monthly Mean Total Sunspot Number", nsamples = 10, 
                    noise=False, plot_samples=False, statistic="Median")

## Modelo entrenado SE

In [ ]:
train(tgp_se)

In [ ]:
tgp_se.plot_predict('Sunspots Gaussian Process', 'Year', 'Yearly Mean of Total Sunspots Number',
                    nsamples = 100, noise=False, plot_samples=False,
                    ylim_by_CI=True)

# Modelo Kernel SM

In [ ]:
y_anual.plot()

In [ ]:
from scipy.signal import periodogram

In [ ]:
px, py = periodogram(y_anual.values)
tg.plt.plot(1/px, py)
tg.plt.xlim([0,15])

In [ ]:
def model_sm(df, index_obs, dim = 100): 
    inputs = ['year']
    outputs = ["Monthly Mean Total Sunspot Number"]
    dt = tg.DataTensor(df[inputs + outputs], inputs=inputs, outputs=outputs)
    dt.calculate_scale(inputs=True, outputs=False, quantile=False)
    
    var_sm = tg.TgPrior('var', ['var'], dim=dim, low=0.0, high=0.05, alpha=4, beta=2)
    relevance = tg.TgPrior('relevance', ['relevance'], dim=dim, low=70, high=80, alpha=2, beta=4) 
    period = tg.TgPrior('period', ['period'], dim=dim, low=10, high=12.5, alpha=2, beta=2)
    kernel = tg.SM(var_sm, relevance, period)
    
    noise = tg.TgPrior('noise', ['noise'], dim=dim, low=0.01, high=0.1, alpha=2, beta=4)
    shift = tg.TgPrior('shift', ['Sunspots'], dim=dim, low=-1, high=1, alpha=2, beta=4)
    power = tg.TgPrior('power', ['Sunspots'], dim=dim, low=0.01, high=1.2, alpha=2, beta=4)
    
    cov = tg.CovarianceTransport(kernel, noise=tg.WN(noise))
    marginal = tg.MarginalTransport(tg.BoxCoxShift(power, shift) )
    positive = tg.MarginalTransport(tg.Relu())
    
    tgp = tg.TGP([cov, marginal, positive], dt=dt)
    
    tgp.obs(index_obs)
    return tgp

tgp_sm = model_sm(S_train, train_index_anual)

tgp_sm.plot_predict('Sunspots Gaussian Process SE', 'year', "Monthly Mean Total Sunspot Number", nsamples = 100, 
                    noise=False, plot_samples=False, statistic="Median")

In [ ]:
train(tgp_sm)

In [ ]:
tgp_sm.plot_predict('Sunspots Gaussian Process SE', 'year', "Monthly Mean Total Sunspot Number", nsamples = 100, 
                    noise=False, plot_samples=False, statistic="Median")

In [ ]:
train(tgp_sm, sample_priors=False)

In [ ]:
tgp_sm.plot_predict('Sunspots Gaussian Process SM', 'year', "Monthly Mean Total Sunspot Number", nsamples = 100, 
                    noise=False, plot_samples=False, statistic="Median")

In [ ]:
tgp_sm.plot_predict('Sunspots Gaussian Process SM', 'year', "Monthly Mean Total Sunspot Number", nsamples = 100, 
                    noise=True, plot_samples=False, statistic="Median")

# Modelo Kernel SM + SM

In [ ]:
y_anual.plot()

In [ ]:
from scipy.signal import periodogram

In [ ]:
px, py = periodogram(y_anual.values, nfft=1000)
tg.plt.figure(1, figsize=(15,8))
tg.plt.plot(1/px, py)
tg.plt.xlim([0,150])

In [ ]:
def model_sm(df, index_obs, dim = 100): 
    inputs = ['year']
    outputs = ["Monthly Mean Total Sunspot Number"]
    dt = tg.DataTensor(df[inputs + outputs], inputs=inputs, outputs=outputs)
    dt.calculate_scale(inputs=True, outputs=False, quantile=False)
    
    var_sm1 = tg.TgPrior('var1', ['var1'], dim=dim, low=0.0, high=0.05, alpha=4, beta=2)
    relevance1 = tg.TgPrior('relevance1', ['relevance1'], dim=dim, low=70, high=80, alpha=2, beta=4) 
    period1 = tg.TgPrior('period1', ['period1'], dim=dim, low=10, high=12.5, alpha=2, beta=2)
    kernel1 = tg.SM(var_sm1, relevance1, period1)
    
    var_sm2 = tg.TgPrior('var2', ['var2'], dim=dim, low=0.0, high=0.05, alpha=4, beta=2)
    relevance2 = tg.TgPrior('relevance2', ['relevance2'], dim=dim, low=70, high=80, alpha=2, beta=4) 
    period2 = tg.TgPrior('period2', ['period2'], dim=dim, low=90, high=120, alpha=2, beta=2)
    kernel2 = tg.SM(var_sm2, relevance2, period2)
    
    
    
    
    noise = tg.TgPrior('noise', ['noise'], dim=dim, low=0.01, high=0.1, alpha=2, beta=4)
    shift = tg.TgPrior('shift', ['Sunspots'], dim=dim, low=-1, high=1, alpha=2, beta=4)
    power = tg.TgPrior('power', ['Sunspots'], dim=dim, low=0.01, high=1.2, alpha=2, beta=4)
    
    cov = tg.CovarianceTransport(kernel1 + kernel2, noise=tg.WN(noise))
    marginal = tg.MarginalTransport(tg.BoxCoxShift(power, shift) )
    positive = tg.MarginalTransport(tg.Relu())
    
    tgp = tg.TGP([cov, marginal, positive], dt=dt)
    
    tgp.obs(index_obs)
    return tgp

tgp_sm = model_sm(S_train, train_index_anual)

tgp_sm.plot_predict('Sunspots Gaussian Process SE', 'year', "Monthly Mean Total Sunspot Number", nsamples = 100, 
                    noise=False, plot_samples=False, statistic="Median")

In [ ]:
train(tgp_sm)

In [ ]:
tgp_sm.plot_predict('Sunspots Gaussian Process SE', 'year', "Monthly Mean Total Sunspot Number", nsamples = 100, 
                    noise=False, plot_samples=False, statistic="Median")

In [ ]:
train(tgp_sm, sample_priors=False)

In [ ]:
tgp_sm.plot_predict('Sunspots Gaussian Process SM', 'year', "Monthly Mean Total Sunspot Number", nsamples = 100, 
                    noise=False, plot_samples=False, statistic="Median")

In [ ]:
tgp_sm.plot_predict('Sunspots Gaussian Process SM', 'year', "Monthly Mean Total Sunspot Number", nsamples = 100, 
                    noise=True, plot_samples=False, statistic="Median")

# Modelo Kernel SM + SM + SE

In [ ]:
y_anual.plot()

In [ ]:
from scipy.signal import periodogram

In [ ]:
px, py = periodogram(y_anual.values, nfft=1000)
tg.plt.figure(1, figsize=(15,8))
tg.plt.plot(1/px, py)
tg.plt.xlim([0,150])

In [ ]:
def model_sm(df, index_obs, dim = 100): 
    inputs = ['year']
    outputs = ["Monthly Mean Total Sunspot Number"]
    dt = tg.DataTensor(df[inputs + outputs], inputs=inputs, outputs=outputs)
    dt.calculate_scale(inputs=True, outputs=False, quantile=False)
    
    var_sm1 = tg.TgPrior('var1', ['var1'], dim=dim, low=0.0, high=0.05, alpha=4, beta=2)
    relevance1 = tg.TgPrior('relevance1', ['relevance1'], dim=dim, low=70, high=80, alpha=2, beta=4) 
    period1 = tg.TgPrior('period1', ['period1'], dim=dim, low=10, high=12.5, alpha=2, beta=2)
    kernel1 = tg.SM(var_sm1, relevance1, period1)
    
    var_sm2 = tg.TgPrior('var2', ['var2'], dim=dim, low=0.0, high=0.05, alpha=4, beta=2)
    relevance2 = tg.TgPrior('relevance2', ['relevance2'], dim=dim, low=70, high=80, alpha=2, beta=4) 
    period2 = tg.TgPrior('period2', ['period2'], dim=dim, low=90, high=120, alpha=2, beta=2)
    kernel2 = tg.SM(var_sm2, relevance2, period2)
    
    var_se = tg.TgPrior('var3', ['var3'], dim=dim, low=0.0, high=0.05, alpha=4, beta=2)
    relevance3 = tg.TgPrior('relevance3', ['relevance3'], dim=dim, low=0, high=10, alpha=2, beta=4) 
    kernel3 = tg.SE(var_se, relevance3)   
    
    noise = tg.TgPrior('noise', ['noise'], dim=dim, low=0.01, high=0.1, alpha=2, beta=4)
    shift = tg.TgPrior('shift', ['Sunspots'], dim=dim, low=-1, high=1, alpha=2, beta=4)
    power = tg.TgPrior('power', ['Sunspots'], dim=dim, low=0.01, high=1.2, alpha=2, beta=4)
    
    cov = tg.CovarianceTransport(kernel1 + kernel2 + kernel3, noise=tg.WN(noise))
    marginal = tg.MarginalTransport(tg.BoxCoxShift(power, shift) )
    positive = tg.MarginalTransport(tg.Relu())
    
    tgp = tg.TGP([cov, marginal, positive], dt=dt)
    
    tgp.obs(index_obs)
    return tgp

tgp_sm = model_sm(S_train, train_index_anual)

tgp_sm.plot_predict('Sunspots Gaussian Process SE', 'year', "Monthly Mean Total Sunspot Number", nsamples = 100, 
                    noise=False, plot_samples=False, statistic="Median")

In [ ]:
train(tgp_sm)

In [ ]:
tgp_sm.plot_predict('Sunspots Gaussian Process SE', 'year', "Monthly Mean Total Sunspot Number", nsamples = 100, 
                    noise=False, plot_samples=False, statistic="Median")

In [ ]:
train(tgp_sm, sample_priors=False)

In [ ]:
tgp_sm.plot_predict('Sunspots Gaussian Process SM', 'year', "Monthly Mean Total Sunspot Number", nsamples = 100, 
                    noise=False, plot_samples=False, statistic="Median")

In [ ]:
tgp_sm.plot_predict('Sunspots Gaussian Process SM', 'year', "Monthly Mean Total Sunspot Number", nsamples = 100, 
                    noise=True, plot_samples=False, statistic="Median")